In [ ]:
!pip install karateclub
!pip install ogb
!pip install node2vec
!pip install dgl
!pip install hypernetx
!pip install celluloid
!pip install igraph

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
from ogb.nodeproppred import DglNodePropPredDataset
import hypernetx as hnx
import networkx as nx
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from karateclub import DeepWalk
from node2vec import Node2Vec

In [ ]:
'''
here, the hyG creation starts
1. at frist we read the text file "new_graph_data_1.txt". This text file is created by running ESPF algorithm
2. Then we create a dictionary 'DICT' that has 'drug' as key and drug's 'substructures' as values 
'''
formattedNodes = []
smileFile = open("drive/My Drive/Colab Notebooks/Hypergraph_my/new_graph_data_824_kmer.txt", 'r') #ESPF or k-mer
names = []

for line in smileFile:
    line = line.strip()
    words = line.split(" ")
    name = words[0]
    del words[0]
    t=tuple(words)
    names.append(name)
    formattedNodes.append(t)
      
# Make the dictionary of the Drug IDs: Smiles tokens    
DICT = dict()  
for i in range(0, len(names)):
    DICT[names[i]] = formattedNodes[i]    



In [ ]:
'''
feature one hot coding
'''
nl=np.zeros((824,824))
np.fill_diagonal(nl, 1)
drug_X=nl

In [ ]:
'''
Then we are using the DICT to create a hyG
'''
drug_list = [] # list of drug - translates drug name into index
chemicalsub_list = [] # list of chemicalsub - translates chemicalsub number into index
drug_chemicalsub = {} # dict of int:list - drug idx i wrote chemicalsubs with idx [j, k, l, .. ]
#chemicalsub_drug = {} # dict of int:list - chemicalsub idx i were written by drugs with idx [j, k, l, ..]

for drug in DICT.keys() :
    chemicalsubs = DICT[drug]
    if drug not in drug_list :
        drug_list.append(drug)
    idx = drug_list.index(drug)
    if idx not in drug_chemicalsub :
        drug_chemicalsub[idx]=[]   
    
    translated_p = []
   
    for chemicalsub in chemicalsubs :
        if chemicalsub not in chemicalsub_list :
            chemicalsub_list.append(chemicalsub)
        p_idx = chemicalsub_list.index(chemicalsub)
        translated_p.append(p_idx) # translate chemicalsub number into index
  
    drug_chemicalsub[idx]=translated_p

'''
chemicalsub : corresponds to hypernode
citing : corresponds to hyperedge(drug)
'''

chemicalsub_citing = [] # list of [chemicalsub drug]
n_chemicalsub = len(chemicalsub_list)
n_hedge = len(drug_list)

for drug in drug_chemicalsub.keys():
    chemicalsubs = drug_chemicalsub[drug]
    #if len(chemicalsubs) == 1 :
        #continue # remove one-cited chemicalsubs
    for chemicalsub in chemicalsubs :
        chemicalsub_citing.append([chemicalsub, drug])
       
chemicalsub_drug = torch.LongTensor(chemicalsub_citing)
data_dict = {
        ('node', 'in', 'edge'): (chemicalsub_drug[:,0], chemicalsub_drug[:,1]),        
        ('edge', 'con', 'node'): (chemicalsub_drug[:,1], chemicalsub_drug[:,0])
    }


'''
finally passing the data_dict to construct hyG
'''
hyG = dgl.heterograph(data_dict)

rows=n_chemicalsub
columns=n_hedge

'''
reading the feature (one hot coding) for drugs (edges)
'''

#drug_X=np.loadtxt("drive/My Drive/Colab Notebooks/Hypergraph_my/features_one_hot_coding_824.txt", dtype=int) #if you want to use 167 bits representation of each drug as feature read: features_bit.txt    
hyG.ndata['h'] = {'edge' : torch.tensor(drug_X).type('torch.FloatTensor'), 'node' : torch.ones(rows, 100)}
e_feat = torch.tensor(drug_X).type('torch.FloatTensor')
v_feat = torch.ones(rows, 100)

In [1]:
hyG

NameError: ignored

In [ ]:
"""
The main regular graph creation (mat) from drugbank data, that will be used for train-test purpose.
1. first we are creating a 2D matrix 'mat'
2. writing the edge information in a text file
3. reading the saved text file and extracting the src and dst node from that for each edge.
These src, dst info will be used in DGL to create a dgl type graph
"""

#1:
df=pd.read_csv("drive/My Drive/Colab Notebooks/Hypergraph_my/drug_and_smiles_824.csv") 
mat = np.zeros((824, 824))
df1=pd.read_csv("drive/My Drive/Colab Notebooks/Hypergraph_my/drugID_to_Integer_Mapping_824.txt", header=None, sep=' ')
D={}
for i in range (len (df1)):
    D[df1[1][i]]=df1[0][i]
for i in range (len(df['Interacting Drugs'])):
    SP=df['Interacting Drugs'][i]
    SP=SP.split('; ')
    SP1=list(SP)
    for j in SP1:
        for drug, id in D.items():
            if j==drug:
                mat[i,id]=mat[id,i]=1

#np.count_nonzero(mat == 1)    

#2:
with open("drive/My Drive/Colab Notebooks/Hypergraph_my/edge_list_regular_graph_824.txt", "w") as file_prime:
  for i in range (len(mat)):
    for j in range (len(mat)):
      if mat[i,j] == 1.0:
        stra=str(i)
        strb=str(j)
        strc=stra+'\t'+strb+'\n'
        file_prime.write(strc)


#3:
src=[]
dst=[]

with open("drive/My Drive/Colab Notebooks/Hypergraph_my/edge_list_regular_graph_824.txt") as fp:
        for i,line in enumerate (fp):
            info = line.strip().split()
            src.append(info[0])
            dst.append(info[1])

src=np.asarray(src, dtype=np.int64) #to use in the data prerpocessing
dst=np.asarray(dst, dtype=np.int64) #to use in the data prerpocessing


In [ ]:
"""
here, we are using src, dst info to create a DGL style graph. this graph (g) will be used for training and testing purpose.

Source: DGL
"""
from dgl.data import DGLDataset
import os
import torch.nn as nn
import torch.nn.functional as F

class KarateClubDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')
    def process(self):
        edges_src = torch.from_numpy(src)
        edges_dst = torch.from_numpy(dst)
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=824)
    def __getitem__(self, i):
        return self.graph
    def __len__(self):
        return 1

dataset = KarateClubDataset()
g = dataset[0]
print(g)

In [ ]:
'''
Train, test (both positive and negative) graphs creation for training and testing purpose from the dgl graph (g)
Source: DGL
'''
# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids=eids.tolist()
test_size = int(len(eids) * 0.3)
train_size = g.number_of_edges() - test_size
test_size1=int(test_size/2)

In [ ]:
T1=[]
T2=[]
for i in eids:
  m, n = int(u[i]), int(v[i])
  c=(m,n)
  T1.append(c)

for i in T1:
  m=i[0]
  n=i[1]
  c=(m,n)
  d=(n,m)
  if c not in T2 and d not in T2:
    T2.append(c)
  if len(T2)>=test_size1:
    break  

test_eids=[]
for i in T2:
  m= i[0]
  n=i[1]
  c=g.edge_id(m,n)
  d=g.edge_id(n,m)
  test_eids.append(c)
  test_eids.append(d)

train_eids=[]
for i in range (len(eids)):
  a=int(eids[i])
  if a not in test_eids:
    train_eids.append(a)  

In [ ]:
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]


train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges())

In [ ]:
g_for_baseline = dgl.remove_edges(g, test_eids)
#dgl.save_graphs("drive/My Drive/Colab Notebooks/Hypergraph_my/a.bin",g_for_baseline)
g_for_baseline = dgl.add_self_loop(g_for_baseline)

In [ ]:
'''
Decoder part. 
2 types of decoders.
Anyone u can use.
Source: DGL
'''

import dgl.function as fn
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h # here h is 822 drug features and g is the pos/neg train/test graph which is nothing but edge lis
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']


decoder = MLPPredictor(100)
#decoder = DotPredictor()# You can replace DotPredictor with MLPPredictor.
#opt = torch.optim.Adam(list(model.parameters()) + list(pred.parameters()))

In [ ]:
'''
compute_loss() and compute_auc() define
'''
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy() 
    precision, recall, thresholds = precision_recall_curve(labels, scores)
    auc_precision_recall = auc(recall, precision)
    return roc_auc_score(labels, scores),auc(recall, precision)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.gat1 = HyperAttn_ne(drug_X.shape[1],64,100,100,0.5)
 
    def forward(self,hyG, v_feat, e_feat,f,l):   
        h = self.gat1(hyG, v_feat, e_feat,f,l)
        return h
model = Model()  

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), decoder.parameters()), lr=0.01)
all_logits = []
for e in range(600):
    # forward
    h=model(hyG, v_feat, e_feat,True,True)
    h=h[1]
    pos_score = decoder(train_pos_g, h)
    neg_score = decoder(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 10 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))
# ----------- 5. check results ------------------------ #

with torch.no_grad():
    pos_score = decoder(test_pos_g, h)
    neg_score = decoder(test_neg_g, h)
    c=compute_auc(pos_score, neg_score)
    print('ROC-AUC: ', c[0],'PR-AUC: ',c[1])


In [ ]:
scores = torch.cat([pos_score, neg_score])
labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])

m1 = tf.keras.metrics.BinaryAccuracy()
m1.update_state(labels,scores)
print(m1.result().numpy())

sig_scores=F.sigmoid(scores)
m2 = tf.keras.metrics.Precision()
m2.update_state(labels,sig_scores)
M2=m2.result().numpy()

m3 = tf.keras.metrics.Recall()
m3.update_state(labels,sig_scores)
M3=m3.result().numpy()

F1=2*(M2*M3)/(M2+M3)
print("Precision:",M2,"Recall:",M3,"F1-score:",F1)